# Azure Tests

## 1. Define LLM

In [1]:
from llmstudio import LLM

# llm = LLM('azure/gpt-4o')
llm = LLM('azure/Meta-Llama-3.1-70B-Instruct')

Running LLMstudio Engine on http://localhost:51234 
Running LLMstudio Tracking on http://localhost:51235 


### 1.1 Function Calling

In [2]:
weather_function = {
    "name": "get_current_weather",
    "description": "Retrieves the current weather information for a specified location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city or location to get the weather for, e.g., 'New York City', 'Paris', 'Tokyo'."
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit", "kelvin"],
                "description": "The unit for temperature. Defaults to 'celsius' if not specified."
            }
        },
        "required": ["location"]
    }
}

In [3]:
llm.chat('Hello how are you?', functions = [weather_function])

Exception: string indices must be integers, not 'str'

In [ ]:
llm.chat('What is the weather in Lisbon?', functions = [weather_function])

### 1.2 Tool Calling

In [ ]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

In [ ]:
llm.chat('Hello how are you?', tools = tools)

In [ ]:
llm.chat('What is the time in Lisbon?', tools = tools)

### 1.3 Langchain Agent

In [ ]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent
from llmstudio.llm.langchain import ChatLLMstudio

# llm = ChatLLMstudio(model_id='azure/gpt-4o', temperature=0)
llm = ChatLLMstudio(model_id='azure/Meta-Llama-3.1-70B-Instruct', temperature=0)

In [ ]:
@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"


def assistant(question: str)->str:
    tools = [get_departure, buy_ticket]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [ ]:
llm.invoke('Hi')

In [ ]:
assistant('When does my train depart? My ticket is 1234')